In [1]:
def iter_query_cog(querry_orth):
    with open(querry_orth) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(';')
            yield([buffer[4],buffer[5]])

In [5]:
def cog_2_g_list(cog_id,cog_dir):
    file_path = f'{cog_dir}/COG_{cog_id}.csv'
    g_list = set()
    with open(file_path) as csvfile:
        next(csvfile)
        for l in csvfile:
            buffer = l.split(',')
            strain = buffer[1]
            g_list.add(strain)
    return g_list

In [7]:
p_dict={}
i=0
for cog_id,p_name in iter_query_cog(query_orth):
    g_list = cog_2_g_list(cog_id,cog_dir)
    p_dict[p_name] = g_list